# Mobile Metrics

Loads and saves webpagetest.org performance tests.

Needs a local installation of the webpagetest tools https://github.com/WPO-Foundation/webpagetest and a mobile device or a key for the webpagetest API.

A complete test may last for up to 2 hours.

Feel free to contact me for help: https://www.quel-media.com/about.html#contact

© Paul Ronga under Apache-2 Licence (see LICENCE.txt).

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
import json
import datetime

In [2]:
# Link to local PHP files
RUN_TEST_URL = 'http://rospo.local/~paul/webpagetest/run_mobile.php'
GET_TEST_URL = 'http://rospo.local/~paul/webpagetest/get.php?test_id={}'

In [3]:
# dataframe containing media id, name and URLs
medias = pd.read_csv('df/media_list.csv')

# media id as string
medias['media_id'] = medias['media_id'].apply(lambda x: str(x))

medias.head(2)

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch
1,20,24 heures,24heures.ch,https://www.24heures.ch,https://m.24heures.ch


# 1. Lauch tests

Lauch tests (average of 3 runs) for each media on a device monitored with the webpagetest system.

In [4]:
reports = {}

In [5]:
for i, row in medias.iterrows():
    url = row['URL_mobile']
    if url != url or url == 'x':
        url = row['URL']

    print('Running test for', url)

    payload = {'url': url}
    r = requests.post(RUN_TEST_URL, json=payload)
    
    print(r.text, end='\n\n')
    result = r.json()
    reports[row['media_id']] = result['id']

Running test for https://m.tdg.ch
{"id":"180624_R0_acb0748e369a8e0ad3675a7980151216", "url":"https://m.tdg.ch"}

Running test for https://m.24heures.ch
{"id":"180624_MW_4840f5590f3f2b11b266abf09c599300", "url":"https://m.24heures.ch"}

Running test for https://www.letemps.ch
{"id":"180624_FY_349af15c32f0aa31afa670d176e7abe8", "url":"https://www.letemps.ch"}

Running test for https://mobile.lemonde.fr
{"id":"180624_9T_e3652291d111468baeffd03c2228731b", "url":"https://mobile.lemonde.fr"}

Running test for https://www.rts.ch/info/
{"id":"180624_9S_e12c1ef201322f102c549df9bfe0280e", "url":"https://www.rts.ch/info/"}

Running test for https://m.20min.ch/ro/
{"id":"180624_ZV_70d506cf37c836194f5006d9564facde", "url":"https://m.20min.ch/ro/"}

Running test for https://m.lematin.ch
{"id":"180624_QN_e4d6a8bd2db53d998b53a82c7dc68d80", "url":"https://m.lematin.ch"}

Running test for https://www.mediapart.fr
{"id":"180624_K2_b3c8123bf323364e717e8fa1c41553a8", "url":"https://www.mediapart.fr"}

Runn

# 2. Get test results

Wait about one hour before trying to get these results.

In [6]:
df_speed = pd.DataFrame(columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url'])

In [18]:
for i, row in medias.iterrows():
    print('Getting results for', row['Name'], '...')
    
    if len(df_speed[df_speed['media_id'] == row['media_id']]) > 0:
        print('Result already fetched.')
    else:    
        r = requests.get(GET_TEST_URL.format( reports[row['media_id']] ))
        print(r)
        try:
            data = r.json()
        except ValueError as e:
            print(e)
            print(r.text)

        missing_key = False

        if 'data' in data.keys() and 'average' in data['data'].keys():    
            print('Successful first view runs:', data['data']['successfulFVRuns'])

            print('Location and device were:', data['data']['location'], 'with connectivity:', data['data']['connectivity'])
            
            for key in ['visualComplete', 'fullyLoaded', 'requests']:
                if key not in data['data']['average']['firstView'].keys():
                    print('Missing key:', key)
                    print(data['data']['average']['firstView'].keys())
                    if key == 'visualComplete' and 'fullyLoaded' in data['data']['average']['firstView'].keys():
                        data['data']['average']['firstView']['visualComplete'] = data['data']['average']['firstView']['fullyLoaded']
                    else:
                        missing_key = True

            if not missing_key:
                df_speed = df_speed.append(pd.DataFrame([[
                    row['Name'],
                    row['media_id'],
                    data['data']['location'],
                    int(data['data']['average']['firstView']['visualComplete']),
                    int(data['data']['average']['firstView']['fullyLoaded']),
                    int(data['data']['average']['firstView']['requests']),
                    data['data']['summary']
                ]], columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url']))
            
                with open('df/archive/mobile/{}.json'.format(data['data']['testId']), 'w') as outfile:
                    json.dump(data['data'], outfile)
                print('Saved to disk and json stored in df/archive/mobile/', end='\n\n')
        else:
            print('No result for {} yet.'.format(row['Name']))

Getting results for La Tribune de Genève ...
Result already fetched.
Getting results for 24 heures ...
Result already fetched.
Getting results for Le Temps ...
Result already fetched.
Getting results for Le Monde ...
Result already fetched.
Getting results for RTS info ...
Result already fetched.
Getting results for 20 minutes (ch) ...
Result already fetched.
Getting results for Le Matin ...
<Response [200]>
Successful first view runs: 2
Location and device were: Dulles_iPhone8:iPhone 8 iOS 11 with connectivity: 4G
Saved to disk and json stored in df/archive/mobile/

Getting results for Mediapart ...
<Response [200]>
Successful first view runs: 3
Location and device were: Dulles_iPhone8:iPhone 8 iOS 11 with connectivity: 4G
Saved to disk and json stored in df/archive/mobile/

Getting results for Le Figaro ...
<Response [200]>
Successful first view runs: 2
Location and device were: Dulles_iPhone8:iPhone 8 iOS 11 with connectivity: 4G
Saved to disk and json stored in df/archive/mobile/



In [19]:
data

{'data': {'startTime': '06/24/18 13:40:25'},
 'statusCode': '101',
 'statusText': 'Test Started'}

In [20]:
df_speed.sort_values('fullyLoaded')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url
0,Libération,28,Dulles_iPhone8:iPhone 8 iOS 11,4839,11972,150,http://www.webpagetest.org/result/180624_EQ_92...
0,Le Monde,22,Dulles_iPhone8:iPhone 8 iOS 11,7351,12882,100,http://www.webpagetest.org/result/180624_9T_e3...
0,24 heures,20,Dulles_iPhone8:iPhone 8 iOS 11,16840,21934,187,http://www.webpagetest.org/result/180624_MW_48...
0,Le Temps,21,Dulles_iPhone8:iPhone 8 iOS 11,12029,22111,157,http://www.webpagetest.org/result/180624_FY_34...
0,Le Matin,25,Dulles_iPhone8:iPhone 8 iOS 11,6276,23101,161,http://www.webpagetest.org/result/180624_QN_e4...
0,Le Courrier,33,Dulles_iPhone8:iPhone 8 iOS 11,23019,23423,70,http://www.webpagetest.org/result/180624_QR_af...
0,RTS info,23,Dulles_iPhone8:iPhone 8 iOS 11,19846,24069,165,http://www.webpagetest.org/result/180624_9S_e1...
0,La Tribune de Genève,19,Dulles_iPhone8:iPhone 8 iOS 11,24301,27173,290,http://www.webpagetest.org/result/180624_R0_ac...
0,20 minutes (ch),24,Dulles_iPhone8:iPhone 8 iOS 11,21646,28737,104,http://www.webpagetest.org/result/180624_ZV_70...
0,Le Nouvelliste,31,Dulles_iPhone8:iPhone 8 iOS 11,31869,33701,123,http://www.webpagetest.org/result/180624_F3_89...


In [28]:
df_speed['mobile_index'] = df_speed['fullyLoaded'].apply(lambda x: round(100 - (int(x)-5000)/1000))
df_speed.sort_values('mobile_index')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url,mobile_index
0,Konbini Suisse,34,Dulles_iPhone8:iPhone 8 iOS 11,15101,86548,87,http://www.webpagetest.org/result/180624_XT_5f...,18
0,Mediapart,26,Dulles_iPhone8:iPhone 8 iOS 11,49687,72429,86,http://www.webpagetest.org/result/180624_K2_b3...,33
0,Le Figaro,27,Dulles_iPhone8:iPhone 8 iOS 11,64722,72394,227,http://www.webpagetest.org/result/180624_CC_fc...,33
0,La Liberté,32,Dulles_iPhone8:iPhone 8 iOS 11,57312,63114,159,http://www.webpagetest.org/result/180624_BE_20...,42
0,Arcinfo,30,Dulles_iPhone8:iPhone 8 iOS 11,45403,50685,129,http://www.webpagetest.org/result/180624_FB_77...,54
0,La Côte,29,Dulles_iPhone8:iPhone 8 iOS 11,36691,47326,144,http://www.webpagetest.org/result/180624_6D_0c...,58
0,Le Nouvelliste,31,Dulles_iPhone8:iPhone 8 iOS 11,31869,33701,123,http://www.webpagetest.org/result/180624_F3_89...,71
0,20 minutes (ch),24,Dulles_iPhone8:iPhone 8 iOS 11,21646,28737,104,http://www.webpagetest.org/result/180624_ZV_70...,76
0,La Tribune de Genève,19,Dulles_iPhone8:iPhone 8 iOS 11,24301,27173,290,http://www.webpagetest.org/result/180624_R0_ac...,78
0,RTS info,23,Dulles_iPhone8:iPhone 8 iOS 11,19846,24069,165,http://www.webpagetest.org/result/180624_9S_e1...,81


In [29]:
outputfile = 'df/archive/mobile_metrics_{}.csv'.format( datetime.datetime.now().strftime('%Y-%m-%d') )
print('Saving to {}...'.format(outputfile))

Saving to df/archive/mobile_metrics_2018-06-24.csv...


In [34]:
df_speed.to_csv(outputfile) # archive
df_speed.to_csv('df/mobile_metrics.csv') # temp file

In [31]:
df = pd.read_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

In [36]:
df_mobile = pd.read_csv('df/mobile_metrics.csv', usecols=['media_id', 'location', 'visualComplete', 'fullyLoaded',
       'requests', 'report_url', 'mobile_index'])

In [37]:
df.merge(df_mobile, on='media_id').to_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

### NB

** This script only uses loading time. Optimization is evaluated with PageSpeed. But these 9 scores are also available: **
```'score_cache',
'score_cdn',
'score_combine',
'score_compress',
'score_cookies',
'score_etags',
'score_gzip',
'score_keep-alive',
'score_minify'```